In [11]:
import numpy as np

In [27]:
# text를 받아서 corpus, word_to_id, id_to_word를 반환하는 함수 생성

def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}

    '''
    Fill in your answer
    1. For문을 이용하여 word_to_id, id_to_word를 만들고
    2. word_to_id를 이용해 corpus 생성
    '''
    for word in words:
        if not word in word_to_id:
            idx = len(word_to_id)
            word_to_id[word] = idx
            id_to_word[idx] = word
    
    corpus = [word_to_id[word] for word in words]
    
    return corpus, word_to_id, id_to_word

preprocess("I am a kid.")

([0, 1, 2, 3, 4],
 {'i': 0, 'am': 1, 'a': 2, 'kid': 3, '.': 4},
 {0: 'i', 1: 'am', 2: 'a', 3: 'kid', 4: '.'})

In [29]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    '''동시발생 행렬 생성

    :param corpus: 말뭉치(단어 ID 목록)
    :param vocab_size: 어휘 수
    :param window_size: 윈도우 크기(윈도우 크기가 1이면 타깃 단어 좌우 한 단어씩이 맥락에 포함)
    :return: 동시발생 행렬
    '''
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            '''
            Fill in your answer
            - left_idx, right_idx 각각 위치에 해당하는 단어 값에 +1
            '''
            
            if left_idx >= 0:
                co_word_id = corpus[left_idx]
                co_matrix[co_word_id][word_id] += 1
                co_matrix[word_id][co_word_id] += 1
            
            if right_idx < corpus_size:
                co_word_id = corpus[right_idx]
                co_matrix[co_word_id][word_id] += 1
                co_matrix[word_id][co_word_id] += 1
            
    return co_matrix

corpus, word_to_id, id_to_word = preprocess("I am a kid.")
C = create_co_matrix(corpus, len(word_to_id), 1)
print(C)

[[0 2 0 0 0]
 [2 0 2 0 0]
 [0 2 0 2 0]
 [0 0 2 0 2]
 [0 0 0 2 0]]


In [16]:
# 유사 단어 검색하는 함수

def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    '''유사 단어 검색

    :param query: 쿼리(텍스트)
    :param word_to_id: 단어에서 단어 ID로 변환하는 딕셔너리
    :param id_to_word: 단어 ID에서 단어로 변환하는 딕셔너리
    :param word_matrix: 단어 벡터를 정리한 행렬. 각 행에 해당 단어 벡터가 저장되어 있다고 가정한다.
    :param top: 상위 몇 개까지 출력할 지 지정
    '''
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return

    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # 코사인 유사도 계산
    vocab_size = len(id_to_word)

    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # 코사인 유사도를 기준으로 내림차순으로 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

In [26]:
def cos_similarity(x, y, eps=1e-8):
    '''코사인 유사도 산출

    :param x: 벡터
    :param y: 벡터
    :param eps: '0으로 나누기'를 방지하기 위한 작은 값
    :return:
    '''

    '''
    Fill in your answer
    - 코사인 유사도의 정의를 이용하여
    '''
    nx = x / (np.sqrt(np.square(x).sum()) + eps)
    ny = y / (np.sqrt(np.square(y).sum()) + eps)
    
    return np.dot(nx, ny)

cos_similarity(np.array([1, 2]), np.array([2, 4]))

0.9999999932917959

In [31]:
def ppmi(C, verbose=False, eps = 1e-8):
    '''PPMI(점별 상호정보량) 생성

    :param C: 동시발생 행렬
    :param verbose: 진행 상황을 출력할지 여부
    :return:
    '''
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            '''
            Fill in your answer
            1. 단어 사이의 pmi를 구하고
            2. 0과 pmi 중 max 값을 반환
            단, log2를 사용
            '''
            pmi = np.log2(N * C[i, j] / (S[i] * S[j] + eps))
            M[i, j] = max(0, pmi)
            if verbose:
                cnt += 1
                if cnt % (total//100 + 1) == 0:
                    print('%.1f%% 완료' % (100*cnt/total))
                    
    return M

ppmi(C)

/data/packages/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log2


array([[0., 2., 0., 0., 0.],
       [2., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0.],
       [0., 0., 1., 0., 2.],
       [0., 0., 0., 2., 0.]], dtype=float32)

In [39]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
import ptb

'''
window_size : 2
wordvec_size = 100
'''

window_size = 2
wordvec_size = 100

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
print('동시발생 수 계산 ...')

'''
Fill in your answer

1. 동시발생행렬 생성
'''
C = create_co_matrix(corpus=corpus, vocab_size=vocab_size)

print('PPMI 계산 ...')
'''
Fill in your answer

1. PPMI 계산
'''
W = ppmi(C)

print('calculating SVD ...')
try:
    # truncated SVD (빠르다!)
    from sklearn.utils.extmath import randomized_svd

    '''
    Fill in your answer
    1. randomized_svd를 사용하여 U, S, V 반환
    단, n_iter : 5, n_components : wordvec_size
    '''
    U, S, V = randomized_svd(W, n_components=wordvec_size, n_iter=5)

except ImportError:
    # SVD (느리다)
    '''
    Fill in your answer
    1. numpy의 svd 함수 사용하여 U, S, V 반환
    '''
    U, S, V = np.linalg.svd(W)

'''
Fill in your answer
1. wordvec_size로 truncate 시키기
'''

word_vecs = U[:, :wordvec_size]

querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

동시발생 수 계산 ...
PPMI 계산 ...


/data/packages/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log2


calculating SVD ...

[query] you
 we: 0.8425182700157166
 i: 0.8331506848335266
 they: 0.655825138092041
 there: 0.5441756248474121
 exactly: 0.5286860466003418

[query] year
 week: 0.8960786461830139
 month: 0.870625376701355
 summer: 0.7156308889389038
 period: 0.71495521068573
 day: 0.7052380442619324

[query] car
 auto: 0.6598967909812927
 truck: 0.603825569152832
 luxury-car: 0.5636757016181946
 fast-growing: 0.5518054962158203
 jewelry: 0.5344263315200806

[query] toyota
 kuwait: 0.6819412112236023
 pakistan: 0.6040564179420471
 aeroflot: 0.5711824297904968
 daimler-benz: 0.5506374835968018
 bullets: 0.5410138368606567
